In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import itertools
import scipy
import os
import nept

from matplotlib import animation, rc
from IPython.display import HTML

from loading_data import get_data
from analyze_tuning_curves import get_tuning_curves
from utils_maze import get_trials
from analyze_decode import get_decoded

In [ ]:
thisdir = os.getcwd()
pickle_filepath = os.path.join(thisdir, "cache", "pickled")
output_filepath = os.path.join(thisdir, "plots", "test")

In [ ]:
import info.r068d6 as info

In [ ]:
events, position, spikes, lfp, lfp_theta = get_data(info)

position = position.time_slice(info.task_times['phase3'].start, info.task_times['phase3'].stop)
spikes = [spiketrain.time_slice(info.task_times['phase3'].start, info.task_times['phase3'].stop) for spiketrain in spikes]

In [ ]:
xedges, yedges = nept.get_xyedges(position, binsize=8)

trial_epochs = get_trials(events, info.task_times['phase3'])

In [ ]:
percents_decoded = []

for trial_idx in range(trial_epochs.n_epochs):
    trial_start = trial_epochs.starts[trial_idx]
    trial_stop = trial_epochs.stops[trial_idx]

    trial_times = nept.Epoch([trial_start, trial_stop])
    neurons = get_tuning_curves(info, position, spikes, xedges, yedges, speed_limit=0.4,
                                phase_id="phase3", trial_times=trial_times, trial_number=trial_idx,
                                cache=False)

    args = dict(min_swr=3,
                min_neurons=2,
                min_spikes=1,
                neurons=neurons,
                info=info,
                normalized=False,
                window=0.025,
                dt=0.025,
                gaussian_std=0.0075,
                speed_limit=0.4,
                run_time=True,
                decode_sequences=False,
                min_proportion_decoded=0.1,
                decoding_times=trial_times,
                shuffle_id=False,
                random_shuffle=False,
                )

    decode = dict()
    decode['decoded'], decode['decoded_epochs'], decode['errors'], decode['position'], decode['likelihood'], decode['percent_decoded'] = get_decoded(**args)
    percents_decoded.append(decode['percent_decoded'])
    
percents_decoded = np.array(percents_decoded)

In [ ]:
cutoff = 50

fig, ax = plt.subplots()
ind = np.arange(len(percents_decoded))
width = 0.8
colours = ['#0072b2', '#009e73',  '#d55e00']

ax.bar(ind, percents_decoded, width, color='k')
ax.axhline(y=cutoff, linestyle=":", color="r")
ax.set_ylabel('Trial number')
ax.set_ylabel('Percent of bins in decoding')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.get_xaxis().tick_bottom()
ax.get_yaxis().tick_left()

In [ ]:
print("Number of trials below cutoff:", np.sum(np.less(percents_decoded, cutoff)))